# LSTM 소개
- 코랩에서 실행해야 함
- 기본 LSTM
- Stacked LSTM
- Bidirectional LSTM


In [6]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

## 시계열 데이터를 테이블 구조로 바꾸는 함수
- n_steps: LSTM 모델이 과거 몇개의 입력을 고려할지를 지정

In [7]:
def split_sequence(sequence, n_steps):
  X, y = [],[]
  for i in range(len(sequence)):
    end_ix = i + n_steps
    if end_ix > len(sequence)-1:
      break
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

- 시계열 데이터를 테이블 구조로 바꾸는 예
- 간단한 데이터로 LSTM 모델을 만드는 방법을 설명하겠다

In [8]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
n_steps = 3
X, y = split_sequence(raw_seq, n_steps)
X, y

(array([[10, 20, 30],
        [20, 30, 40],
        [30, 40, 50],
        [40, 50, 60],
        [50, 60, 70],
        [60, 70, 80]]),
 array([40, 50, 60, 70, 80, 90]))

In [15]:
import numpy as np
x=np.arange(20).reshape(-1,2)
split_sequence(x,4)

X,y = split_sequence(x,4)
X.shape
X

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7]],

       [[ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9]],

       [[ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11]],

       [[ 6,  7],
        [ 8,  9],
        [10, 11],
        [12, 13]],

       [[ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15]],

       [[10, 11],
        [12, 13],
        [14, 15],
        [16, 17]]])

### LSTM 모델은 [samples, timesteps, features] 구조의 입력을 사용
- 2차원이 아니라, 3차원 구조의 데이터가 필요하다

- samples: 샘플수
- timesteps: n_steps를 말함
- features: 한 입력의 특성수

In [10]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
X

array([[[10],
        [20],
        [30]],

       [[20],
        [30],
        [40]],

       [[30],
        [40],
        [50]],

       [[40],
        [50],
        [60]],

       [[50],
        [60],
        [70]],

       [[60],
        [70],
        [80]]])


# 기본 LSTM

In [2]:
model = Sequential()
model.add(LSTM(20, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1)) #출력 DENSE가 1개 (Y)
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=200, verbose=0)

x_input = array([70, 80, 90])

# LSTM은 3차원 이상의 구조를 필요로 한다
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

NameError: ignored

In [ ]:
x_input

array([[[70],
        [80],
        [90]]])

In [1]:
# 샘플 입력 예
x_input = array([700, 800, 900])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

NameError: ignored

# Stacked LSTM

- LSTM을 2개 이상의 레이어로 구성하여 더 복잡한 내부 상태를 모델링
- 첫번째 레이어에서 return_sequences=True 옵션을 지정해야 한다

In [ ]:
model = Sequential()
model.add(LSTM(20, activation='relu', return_sequences=True,
               input_shape=(n_steps, n_features)))
model.add(LSTM(20, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=200, verbose=0)
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[102.854485]]


# Bidirectional LSTM
- 과거 값으로부터 미래를 예측할 뿐 아니라, 미래 값을 보고 과거를 예측하는 기능을 추가
- 양방향으로 학습하며, 계산량이 2배 늘어난다 (파라미터를 2배 사용)


In [ ]:
from keras.layers import Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(20, activation='relu'),
                        input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=200, verbose=0)
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[101.38728]]
